## 事前準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp100_2025/ja/

import os

current_directory = os.getcwd()
print(f"The Current Directory: {current_directory}")

# 第4章: 言語解析

問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

In [ ]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [ ]:
# 事前準備
!uv pip install mecab-python3
!uv pip install pandas
!uv pip install unidic
!python3 -m unidic download
!uv pip install japanize-matplotlib

## 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [ ]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    for parsed_line in mecab.parse(line).splitlines():
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "動詞":
                print("文中の動詞-> ", parsed_line.split("\t")[0])
                print(parsed_line.split("\t")[1])
                print("")

## 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [ ]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    for parsed_line in mecab.parse(line).splitlines():
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "動詞":
                print("文中の動詞-> ", parsed_line.split("\t")[0])
                print(parsed_line.split("\t")[1])
                print("動詞の原形-> ", parsed_line.split("\t")[1].split(",")[10])
                print("")

## 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

In [ ]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    parsed_list = mecab.parse(line).splitlines()
    for index, parsed_line in enumerate(parsed_list):
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "助詞":
                if parsed_line.split("\t")[0] == "の":
                    noun1 = parsed_list[index-1].split("\t")[0]
                    particle = parsed_line.split("\t")[1].split(",")[10]
                    noun2 = parsed_list[index+1].split("\t")[0]
                    print(f"名詞句-> {noun1}{particle}{noun2}")

## 33. 係り受け解析

文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [ ]:
!uv pip install ginza ja_ginza

In [ ]:
import spacy

try:
    nlp = spacy.load('ja_ginza')
except OSError:
    # Fallback or specific model if ja_ginza is not directly available
    # For example, if you downloaded 'ja_core_news_sm'
    nlp = spacy.load('ja_core_news_sm')

# Process the text
doc = nlp(text)

print("係り元トークン\t係り先トークン")
print("----------------\t----------------")

for sent in doc.sents:
    for token in sent:
        if token.head != token: # Avoid printing self-dependencies for the root
            print(f"{token.text}\t{token.head.text}")
    print("--- (End of Sentence) ---")

## 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

In [ ]:
import spacy

# 日本語の言語モデルをロードします (GiNZA または spaCy の日本語モデル)
# 'ja_ginza' をインストールした場合、 'ja_ginza' を使用できます
# 'ja_core_news_sm' のような特定のモデルをダウンロードした場合は、それを使用します
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    # ja_ginza が直接利用できない場合のフォールバックまたは特定のモデル
    # 例: 'ja_core_news_sm' をダウンロードした場合
    nlp = spacy.load('ja_core_news_sm')

# テキストを処理します
doc = nlp(text)

print("「メロス」が主語の場合の述語:")
print("--------------------------")

# 文書内の各トークンに対して反復処理を行います
# 「メロス」を主語としてより洗練された検索を行うアプローチ
for token in doc:
    # トークンのテキストが「メロス」で、かつその依存関係ラベルが主語を示すものであるかを確認します
    if token.text == "メロス" and token.dep_ in ("nsubj", "nsubj:outer"):
        # token.head が述語の主要部（多くは動詞や形容詞）です
        predicate_head = token.head

        # 述語句を構成するため、述語の頭部から開始します
        predicate_phrase = predicate_head.text

        # 述語の頭部に続く助動詞や助詞（右方の子要素）を取得して述語句に追加します
        # これにより、例えば「激怒した」のような句を捉えることができます
        for right_token in predicate_head.rights: # head の右側の子要素をチェック
            if right_token.dep_ in ("aux", "mark"): # 一般的な助動詞やマーカー(助詞など)の依存関係
                predicate_phrase += right_token.text

        print(f"メロス (主語) -> {predicate_phrase} (述語の原形: {predicate_head.lemma_})")

## 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

In [ ]:
!uv pip install spacy ginza ja_ginza matplotlib

In [ ]:
import spacy
from spacy import displacy
from IPython.display import HTML, display # こちらを明示的にインポート

# 日本語の言語モデルをロードします
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    nlp = spacy.load('ja_core_news_sm') # モデルがない場合は適宜変更してください

# 対象の文
sentence = "メロスは激怒した。"

# テキストを処理します
doc = nlp(sentence)

# 係り受け木をHTML文字列として取得します
html = displacy.render(doc, style='dep', jupyter=False, options={'distance': 100})

# IPython.display を使ってHTMLを表示します
print(f"「{sentence}」の係り受け木:")
display(HTML(html)) # ここで display を使用

# もしSVGとして保存したい場合は、以下のようにします：
# svg_output = displacy.render(doc, style='dep', page=False, minify=True, options={'distance':100}) # page=False, minify=True はSVG出力時の一般的なオプション
# with open("dependency_tree.svg", "w", encoding="utf-8") as f:
#     f.write(svg_output)
# print("係り受け木を dependency_tree.svg として保存しました。")

## 36. 単語の出現頻度

問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

In [ ]:
!uv pip install tqdm

In [ ]:
import gzip
import json
import re
from collections import Counter
import spacy
from tqdm.auto import tqdm # tqdm をインポート (Jupyter Notebook用)

# GiNZAまたは適切な日本語モデルをロード
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    nlp = spacy.load('ja_core_news_sm') # 適宜変更

# --- 1. ファイルの読み込み ---
filepath = '../data/jawiki-country.json' # ユーザーのエラーメッセージに合わせたパス

# --- 2. マークアップの除去 ---
def remove_markup(text):
    if not text:
        return ""
    text = re.sub(r"'{2,5}(.+?)'{2,5}", r"\1", text)
    text = re.sub(r"\[\[(?:[^|\]]*?\|)*?([^|\]]+?)\]\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\s([^\]]+?)\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\]", "", text)
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    text = re.sub(r"<.+?>", "", text, flags=re.DOTALL)
    text = re.sub(r"#REDIRECT \[\[(.*?)\]\]", r"\1", text)
    text = re.sub(r"^\*+\s*", "", text, flags=re.MULTILINE)
    text = re.sub(r"<ref(?:[^<>]|\s)*?(?:/>|</ref>)", "", text, flags=re.DOTALL)
    text = re.sub(r"<references\s*/>", "", text, flags=re.DOTALL)
    text = re.sub(r"", "", text, flags=re.DOTALL)
    return text

# --- 3 & 4. 全記事の形態素解析と出現頻度の集計 ---
word_counts = Counter()
articles_processed = 0
TOKENIZER_BYTE_LIMIT = 49149

print(f"'{filepath}' から記事を読み込み、形態素解析と単語頻度の集計を開始します...")

# ファイルの総行数を取得 (プログレスバーのtotalに使用)
total_lines = 0
try:
    with open(filepath, 'r', encoding='utf-8') as f_count:
        for _ in f_count:
            total_lines += 1
    if total_lines == 0:
        print("警告: ファイルが空であるか、読み込めませんでした。")
except FileNotFoundError:
    print(f"エラー: ファイル '{filepath}' が見つかりません。処理を中断します。")
    total_lines = -1 # エラーフラグ
except Exception as e:
    print(f"ファイルの行数カウント中にエラー: {e}")
    total_lines = -1 # エラーフラグ


if total_lines > 0:
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            # tqdmでラップしてプログレスバーを表示 (descは説明文, totalは総数)
            for i, line in tqdm(enumerate(f), total=total_lines, desc="記事処理中"):
                try:
                    article = json.loads(line)
                    title = article.get('title', '不明なタイトル')
                    text_with_markup = article.get('text', '')

                    plain_text = remove_markup(text_with_markup)

                    if plain_text.strip():
                        chunks = plain_text.split('\n\n')
                        for chunk_num, chunk in enumerate(chunks):
                            if not chunk.strip():
                                continue
                            sub_chunks = chunk.split('\n')
                            for sub_chunk_num, sub_chunk in enumerate(sub_chunks):
                                if not sub_chunk.strip():
                                    continue
                                if len(sub_chunk.encode('utf-8')) > TOKENIZER_BYTE_LIMIT:
                                    # print(f"警告: 記事「{title}」のサブチャンク ({chunk_num}-{sub_chunk_num}) が長すぎます。スキップします。") # 詳細すぎるので抑制
                                    continue
                                doc = nlp(sub_chunk)
                                for token in doc:
                                    word_counts[token.lemma_] += 1

                    articles_processed += 1
                    # tqdmが更新するので、定期的なprintは不要になることが多い
                    # if articles_processed % 100 == 0:
                    #     print(f"{articles_processed} 件の記事を処理完了...")


                except json.JSONDecodeError:
                    tqdm.write(f"警告: {i+1}行目のJSONデータのデコードに失敗しました。スキップします。") # tqdm.writeを使うとバーを壊さない
                except Exception as e:
                    tqdm.write(f"警告: {i+1}行目の記事「{title}」処理中にエラーが発生しました: {e}")

    except FileNotFoundError:
        # この部分はファイルの行数カウントで既にチェックされているが、念のため
        print(f"エラー: ファイル '{filepath}' が見つかりませんでした。")
        word_counts = Counter()
    except Exception as e:
        print(f"ファイルの読み込みまたは処理中に予期せぬエラーが発生しました: {e}")
        word_counts = Counter()

if articles_processed > 0:
    print(f"\n全 {articles_processed} 件の記事の処理が完了しました。")
    print("\n単語の出現頻度 上位20語 (基本形):")
    for word, count in word_counts.most_common(20):
        print(f"{word}: {count}")
elif total_lines == 0: # ファイルは存在したが空だった場合
    print("処理対象の記事がファイル内にありませんでした。")
elif total_lines == -1: # ファイルが見つからなかったか、行数カウントでエラー
    print("ファイル処理エラーのため、結果はありません。")
else: # total_lines が 0 で、articles_processed も 0 (正常にループしたが処理対象なし)
    print("処理できる記事がありませんでした。")

## 37. 名詞の出現頻度
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

In [ ]:
import gzip
import json
import re
from collections import Counter
import spacy
from tqdm.auto import tqdm # tqdm をインポート (Jupyter Notebook用)

# GiNZAまたは適切な日本語モデルをロード
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    nlp = spacy.load('ja_core_news_sm') # 適宜変更

# --- 1. ファイルの読み込み ---
# ユーザーの環境に合わせてファイルパスを確認してください。
# 前回の実行で '../data/jawiki-country.json' を使用していたので、それを継続します。
filepath = '../data/jawiki-country.json'

# --- 2. マークアップの除去 ---
def remove_markup(text):
    if not text:
        return ""
    text = re.sub(r"'{2,5}(.+?)'{2,5}", r"\1", text)
    text = re.sub(r"\[\[(?:[^|\]]*?\|)*?([^|\]]+?)\]\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\s([^\]]+?)\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\]", "", text)
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    text = re.sub(r"<.+?>", "", text, flags=re.DOTALL)
    text = re.sub(r"#REDIRECT \[\[(.*?)\]\]", r"\1", text)
    text = re.sub(r"^\*+\s*", "", text, flags=re.MULTILINE)
    text = re.sub(r"<ref(?:[^<>]|\s)*?(?:/>|</ref>)", "", text, flags=re.DOTALL)
    text = re.sub(r"<references\s*/>", "", text, flags=re.DOTALL)
    text = re.sub(r"", "", text, flags=re.DOTALL)
    return text

# --- 3 & 4. 全記事の形態素解析と「名詞」の出現頻度の集計 ---
noun_counts = Counter() # 名詞の頻度を格納するCounter
articles_processed = 0
TOKENIZER_BYTE_LIMIT = 49149

print(f"'{filepath}' から記事を読み込み、形態素解析と「名詞」の頻度の集計を開始します...")

# ファイルの総行数を取得 (プログレスバーのtotalに使用)
total_lines = 0
try:
    with open(filepath, 'r', encoding='utf-8') as f_count:
        for _ in f_count:
            total_lines += 1
    if total_lines == 0:
        print("警告: ファイルが空であるか、読み込めませんでした。")
except FileNotFoundError:
    print(f"エラー: ファイル '{filepath}' が見つかりません。処理を中断します。")
    total_lines = -1 # エラーフラグ
except Exception as e:
    print(f"ファイルの行数カウント中にエラー: {e}")
    total_lines = -1 # エラーフラグ

if total_lines > 0:
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for i, line in tqdm(enumerate(f), total=total_lines, desc="名詞カウント中"):
                try:
                    article = json.loads(line)
                    title = article.get('title', '不明なタイトル')
                    text_with_markup = article.get('text', '')

                    plain_text = remove_markup(text_with_markup)

                    if plain_text.strip():
                        chunks = plain_text.split('\n\n')
                        for chunk_num, chunk in enumerate(chunks):
                            if not chunk.strip():
                                continue
                            sub_chunks = chunk.split('\n')
                            for sub_chunk_num, sub_chunk in enumerate(sub_chunks):
                                if not sub_chunk.strip():
                                    continue
                                if len(sub_chunk.encode('utf-8')) > TOKENIZER_BYTE_LIMIT:
                                    # tqdm.write(f"警告: 記事「{title}」のサブチャンク長すぎスキップ") # 詳細すぎるログは抑制
                                    continue

                                doc = nlp(sub_chunk)
                                for token in doc:
                                    # 品詞が名詞 (NOUN: 普通名詞, PROPN: 固有名詞) の場合のみカウント
                                    if token.pos_ in ['NOUN', 'PROPN']:
                                        noun_counts[token.lemma_] += 1 # 名詞の基本形をカウント
                                        # noun_counts[token.text] += 1 # 表層形をカウントする場合はこちら

                    articles_processed += 1

                except json.JSONDecodeError:
                    tqdm.write(f"警告: {i+1}行目のJSONデコード失敗。スキップします。")
                except Exception as e:
                    tqdm.write(f"警告: {i+1}行目記事「{title}」処理中エラー: {e}")

    except FileNotFoundError:
        print(f"エラー: ファイル '{filepath}' が見つかりませんでした。")
        noun_counts = Counter()
    except Exception as e:
        print(f"ファイル読み込み/処理中エラー: {e}")
        noun_counts = Counter()

if articles_processed > 0:
    print(f"\n全 {articles_processed} 件の記事の処理が完了しました。")
    # --- 5. 上位20語の名詞の表示 ---
    print("\n「名詞」の出現頻度 上位20語 (基本形):")
    for noun, count in noun_counts.most_common(20):
        print(f"{noun}: {count}")
elif total_lines == 0:
    print("処理対象の記事がファイル内にありませんでした。")
elif total_lines == -1:
    print("ファイル処理エラーのため、結果はありません。")
else:
    print("処理できる記事がありませんでした。")

## 38. TF・IDF
日本に関する記事における名詞のTF・IDFスコアを求め、TF・IDFスコア上位20語とそのTF, IDF, TF・IDFを表示せよ。

In [ ]:
import json
import re
from collections import Counter
import spacy
from tqdm.auto import tqdm
import math # IDF計算のため

# GiNZAまたは適切な日本語モデルをロード
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    nlp = spacy.load('ja_core_news_sm')

filepath = '../data/jawiki-country.json' # ユーザーの環境に合わせてファイルパスを確認
TOKENIZER_BYTE_LIMIT = 49149

def remove_markup(text):
    if not text: return ""
    text = re.sub(r"'{2,5}(.+?)'{2,5}", r"\1", text)
    text = re.sub(r"\[\[(?:[^|\]]*?\|)*?([^|\]]+?)\]\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\s([^\]]+?)\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\]", "", text)
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    text = re.sub(r"<.+?>", "", text, flags=re.DOTALL)
    text = re.sub(r"^\*+\s*", "", text, flags=re.MULTILINE)
    text = re.sub(r"<ref(?:[^<>]|\s)*?(?:/>|</ref>)", "", text, flags=re.DOTALL)
    text = re.sub(r"<references\s*/>", "", text, flags=re.DOTALL)
    text = re.sub(r"", "", text, flags=re.DOTALL)
    return text

print(f"'{filepath}' から記事を読み込み、TF-IDF計算のための前処理を開始します...")

# --- ステップ1: 全記事の名詞とその出現回数、DFを収集 ---
# all_doc_noun_counts: 各記事の名詞のCounterを格納するリスト
# doc_freq: 各名詞がいくつの文書に出現したか (Document Frequency)
all_doc_noun_counts = []
doc_freq = Counter()
titles_list = [] # IDF計算時の総文書数Nと、「日本」の記事のインデックス特定のため

total_lines = 0
try:
    with open(filepath, 'r', encoding='utf-8') as f_count:
        for _ in f_count:
            total_lines += 1
except FileNotFoundError:
    print(f"エラー: ファイル '{filepath}' が見つかりません。")
    total_lines = -1

if total_lines > 0:
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in tqdm(f, total=total_lines, desc="全記事の前処理中"):
                article = json.loads(line)
                titles_list.append(article.get('title', ''))
                text_with_markup = article.get('text', '')
                plain_text = remove_markup(text_with_markup)

                current_doc_nouns = Counter()
                if plain_text.strip():
                    chunks = plain_text.split('\n\n')
                    for chunk in chunks:
                        if not chunk.strip(): continue
                        sub_chunks = chunk.split('\n')
                        for sub_chunk in sub_chunks:
                            if not sub_chunk.strip(): continue
                            if len(sub_chunk.encode('utf-8')) > TOKENIZER_BYTE_LIMIT: continue

                            doc_chunk = nlp(sub_chunk)
                            for token in doc_chunk:
                                if token.pos_ in ['NOUN', 'PROPN']:
                                    current_doc_nouns[token.lemma_] += 1

                all_doc_noun_counts.append(current_doc_nouns)
                # DFの更新 (その文書に出現したユニークな名詞をカウント)
                for noun in current_doc_nouns.keys():
                    doc_freq[noun] += 1

    except Exception as e:
        print(f"処理中にエラーが発生しました: {e}")
else:
    print("処理対象ファイルが見つからないか空です。")

if not all_doc_noun_counts:
    print("有効な記事が処理されませんでした。TF-IDF計算をスキップします。")
else:
    # --- ステップ2: 「日本」の記事の特定 ---
    target_article_title = "日本"
    target_doc_index = -1
    try:
        target_doc_index = titles_list.index(target_article_title)
    except ValueError:
        print(f"エラー: 記事「{target_article_title}」が見つかりませんでした。")
        target_doc_index = -1

    if target_doc_index != -1:
        print(f"\n記事「{target_article_title}」のTF-IDFを計算します...")
        target_doc_nouns = all_doc_noun_counts[target_doc_index]
        total_nouns_in_target_doc = sum(target_doc_nouns.values())

        if total_nouns_in_target_doc == 0:
            print(f"記事「{target_article_title}」には名詞が含まれていません。")
        else:
            # --- ステップ3, 4, 5: TF, IDF, TF-IDF の計算 ---
            tfidf_scores = {}
            num_total_docs = len(all_doc_noun_counts) # N: 総文書数

            for noun, tf_count in target_doc_nouns.items():
                # TFの計算
                tf = tf_count / total_nouns_in_target_doc

                # IDFの計算
                # N / (DF(t) + 1) : +1 はDFが0の場合やスムージングのため
                df = doc_freq.get(noun, 0) # もしdoc_freqにない場合は0 (理論上ありえないはずだが安全のため)
                idf = math.log(num_total_docs / (df + 1)) # 自然対数
                # スムージングなしで、DFが0の単語を除外する場合
                # if df > 0:
                #     idf = math.log(num_total_docs / df) + 1 # 別のIDF計算式
                # else:
                #     idf = math.log(num_total_docs / 1) + 1 # DFが0の単語のIDF（例）

                tfidf = tf * idf
                tfidf_scores[noun] = {'tf': tf, 'idf': idf, 'tfidf': tfidf}

            # --- ステップ6: 上位20語の表示 ---
            # TF-IDFスコアでソート
            sorted_tfidf = sorted(tfidf_scores.items(), key=lambda item: item[1]['tfidf'], reverse=True)

            print(f"\n記事「{target_article_title}」における名詞のTF-IDFスコア上位20:")
            print("----------------------------------------------------------")
            print(f"{'名詞':<15} {'TF':<10} {'IDF':<10} {'TF-IDF':<10}")
            print("----------------------------------------------------------")
            for noun, scores in sorted_tfidf[:20]:
                print(f"{noun:<15} {scores['tf']:.4f}     {scores['idf']:.4f}     {scores['tfidf']:.4f}")
    else:
        if titles_list: # titles_listが空でない（＝記事はあった）が「日本」がなかった場合
             print(f"記事リストに「{target_article_title}」が含まれていませんでした。利用可能なタイトル例: {titles_list[:5]}")

## 39. Zipfの法則
コーパスにおける単語の出現頻度順位を横軸、その出現頻度を縦軸として、両対数グラフをプロットせよ。

In [ ]:
!uv pip install japanize-matplotlib

In [ ]:
import json
import re
from collections import Counter
import spacy
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語フォント設定のため

# --- (36番の処理: word_counts を得るまで) ---
# この部分は36番のコードを再利用します。
# word_counts が既に計算済みであれば、このセクションはスキップできます。
# もし未計算の場合は、36番のコードを実行して word_counts を作成してください。

# GiNZAまたは適切な日本語モデルをロード
try:
    nlp = spacy.load('ja_ginza')
except OSError:
    nlp = spacy.load('ja_core_news_sm')

filepath = '../data/jawiki-country.json' # ユーザーの環境に合わせてファイルパスを確認
TOKENIZER_BYTE_LIMIT = 49149

def remove_markup(text):
    if not text: return ""
    text = re.sub(r"'{2,5}(.+?)'{2,5}", r"\1", text)
    text = re.sub(r"\[\[(?:[^|\]]*?\|)*?([^|\]]+?)\]\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\s([^\]]+?)\]", r"\1", text)
    text = re.sub(r"\[https?://[^\s]+?\]", "", text)
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    text = re.sub(r"<.+?>", "", text, flags=re.DOTALL)
    text = re.sub(r"^\*+\s*", "", text, flags=re.MULTILINE)
    text = re.sub(r"<ref(?:[^<>]|\s)*?(?:/>|</ref>)", "", text, flags=re.DOTALL)
    text = re.sub(r"<references\s*/>", "", text, flags=re.DOTALL)
    text = re.sub(r"", "", text, flags=re.DOTALL)
    return text

# word_counts がこのスコープで利用可能か確認
# もし前のセルで計算済みなら、再計算は不要
if 'word_counts' not in locals() or not isinstance(word_counts, Counter) or not word_counts:
    print("word_counts が未計算または空です。36番の処理を実行して単語頻度を計算します...")
    word_counts = Counter()
    articles_processed = 0

    total_lines = 0
    try:
        with open(filepath, 'r', encoding='utf-8') as f_count:
            for _ in f_count:
                total_lines += 1
    except FileNotFoundError:
        print(f"エラー: ファイル '{filepath}' が見つかりません。")
        total_lines = -1

    if total_lines > 0:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                for i, line in tqdm(enumerate(f), total=total_lines, desc="単語頻度計算中 (36番相当)"):
                    try:
                        article = json.loads(line)
                        title = article.get('title', '不明なタイトル')
                        text_with_markup = article.get('text', '')
                        plain_text = remove_markup(text_with_markup)

                        if plain_text.strip():
                            chunks = plain_text.split('\n\n')
                            for chunk in chunks:
                                if not chunk.strip(): continue
                                sub_chunks = chunk.split('\n')
                                for sub_chunk in sub_chunks:
                                    if not sub_chunk.strip(): continue
                                    if len(sub_chunk.encode('utf-8')) > TOKENIZER_BYTE_LIMIT: continue
                                    doc_chunk = nlp(sub_chunk)
                                    for token in doc_chunk:
                                        word_counts[token.lemma_] += 1 # 基本形をカウント
                        articles_processed += 1
                    except json.JSONDecodeError:
                        tqdm.write(f"警告(36): {i+1}行目JSONデコード失敗。スキップ")
                    except Exception as e:
                        tqdm.write(f"警告(36): {i+1}行目記事「{title}」処理中エラー: {e}")
            print(f"全 {articles_processed} 件の記事の単語頻度計算が完了しました。")
        except FileNotFoundError:
             print(f"エラー(36): ファイル '{filepath}' が見つかりませんでした。")
             word_counts = Counter() # 念のため空に
        except Exception as e:
            print(f"エラー(36): ファイル処理中にエラー: {e}")
            word_counts = Counter() # 念のため空に
    else:
        print("処理対象ファイルが見つからないか空です(36)。Zipfのプロットはできません。")

# --- 39. Zipfの法則のプロット ---
if word_counts: # word_counts にデータがある場合のみプロット
    # 1. 単語の出現頻度を取得し、降順にソート
    frequencies = [count for word, count in word_counts.most_common()]

    # 2. 順位を生成 (1位, 2位, 3位, ...)
    ranks = range(1, len(frequencies) + 1)

    # 3. 両対数グラフのプロット
    plt.figure(figsize=(10, 6))
    plt.plot(ranks, frequencies, marker='.', linestyle='none') # 各点をプロット

    plt.xscale('log') # 横軸を対数スケールに
    plt.yscale('log') # 縦軸を対数スケールに

    plt.xlabel('出現頻度順位 (log scale)')
    plt.ylabel('出現頻度 (log scale)')
    plt.title('Zipfの法則: 単語の出現頻度と順位 (両対数グラフ)')
    plt.grid(True)
    plt.show()
else:
    print("単語の出現頻度データがありません。グラフをプロットできません。")